# **Modeling and Evaluation**

## Objectives

#### Business requirement

  - The client is interested in predicting if a cherry leaf is healthy or contains powdery mildew.

#### Outputs

- Images distribution plot in train, validation, and test set
- Image augmentation
- Class indices to change prediction inference in labels
- Machine learning model creation and training
- Save model
- Learning curve plot for model performance
- Model evaluation on pickle file
- Prediction on the random image file
